### Spark for Machine Learning & AI
### 04 Classification

In [2]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.AssertionError: assertion failed: Expected hostname (not IP) but got fe80:0:0:0:aede:48ff:fe00:1122%5
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:1030)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:83)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:60)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:129)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:196)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:570)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstr

In [2]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

### prepocessing

In [3]:
iris_df = spark.read.csv("./iris.data", inferSchema=True)
iris_df.take(1)

[Row(_c0=5.1, _c1=3.5, _c2=1.4, _c3=0.2, _c4='Iris-setosa')]

In [4]:
iris_df=iris_df.select(col("_c0").alias("sepal_length"),
                       col("_c1").alias("sepal_width"),
                       col("_c2").alias("petal_length"),
                       col("_c3").alias("petal_width"),
                       col("_c4").alias("species")
                      )
iris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa')]

In [5]:
vectorAssembler=VectorAssembler(inputCols=["sepal_length","sepal_width","petal_length","petal_width"],
                                outputCol="features")
viris_df=vectorAssembler.transform(iris_df)
viris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [6]:
indexer=StringIndexer(inputCol="species",outputCol="label")
iviris_df=indexer.fit(viris_df).transform(viris_df)
iviris_df.show(1)

+------------+-----------+------------+-----------+-----------+-----------------+-----+
|sepal_length|sepal_width|petal_length|petal_width|    species|         features|label|
+------------+-----------+------------+-----------+-----------+-----------------+-----+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|  0.0|
+------------+-----------+------------+-----------+-----------+-----------------+-----+
only showing top 1 row



### Naive Bayes Classification

In [7]:
iviris_df

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, species: string, features: vector, label: double]

In [8]:
iviris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [9]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [10]:
splits=iviris_df.randomSplit([0.6,0.4],1)
train_df=splits[0]
test_df=splits[1]

In [11]:
train_df.count()

98

In [12]:
test_df.count()

52

In [13]:
iviris_df.count()

150

In [14]:
nb=NaiveBayes(modelType="multinomial")
nbmodel=nb.fit(train_df)
predictions_df=nbmodel.transform(test_df)
predictions_df.take(1)

[Row(sepal_length=4.3, sepal_width=3.0, petal_length=1.1, petal_width=0.1, species='Iris-setosa', features=DenseVector([4.3, 3.0, 1.1, 0.1]), label=0.0, rawPrediction=DenseVector([-9.9894, -11.3476, -11.902]), probability=DenseVector([0.7118, 0.183, 0.1051]), prediction=0.0)]

In [16]:
evaluator=MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
nbaccuracy=evaluator.evaluate(predictions_df)
nbaccuracy

0.9807692307692307

### Multilayer perceptron classification

In [26]:
iviris_df

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, species: string, features: vector, label: double]

In [27]:
iviris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [28]:
train_df.count()

98

In [29]:
test_df.count()

52

In [30]:
iviris_df.count()

150

In [31]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [32]:
#first layer is 4 nerons (4 features), 5 neron in each layer in between, 3 outputs (3 classes) 
layers=[4,5,5,3]

In [33]:
mlp=MultilayerPerceptronClassifier(layers=layers,seed=1)
mlp_model=mlp.fit(train_df)
mlp_predictions=mlp_model.transform(test_df)
mlp_evaluator=MulticlassClassificationEvaluator(metricName="accuracy")
mlp_accuracy=mlp_evaluator.evaluate(mlp_predictions)
mlp_accuracy

0.6923076923076923

### Decision trees classification

In [37]:
iviris_df

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, species: string, features: vector, label: double]

In [38]:
iviris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [39]:
from pyspark.ml.classification import DecisionTreeClassifier

In [44]:
dt=DecisionTreeClassifier(labelCol="label",featuresCol="features")
dt_model=dt.fit(train_df)
dt_predictions=dt_model.transform(test_df)
dt_evaluator=MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
dt_accuracy=dt_evaluator.evaluate(dt_predictions)
dt_accuracy

0.9423076923076923